In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
import warnings
#Removing warnings
warnings.filterwarnings("ignore", category=FutureWarning)


In [2]:
# Load the training features dataset
train_features = pd.read_csv('dengue_features_train.csv')

# Load the training labels dataset
train_labels = pd.read_csv('dengue_labels_train.csv')

# Load the test features dataset
test_features = pd.read_csv('dengue_features_test.csv')

# Load the submission dataset
test_submission = pd.read_csv('submission_format.csv')

# Removing limits on the number of columns
pd.set_option('display.max_columns', None)

FileNotFoundError: [Errno 2] No such file or directory: 'submission_format.csv'

In [ ]:
print(train_features.shape,train_labels.shape, test_features.shape)

In [ ]:
# Display the first few rows of the train_features dataset
train_features.head()

In [ ]:
# Display the first few rows of the train_labels dataset
train_labels.head()

In [ ]:
# Display the first few rows of the test_features dataset
test_features.head()

In [ ]:
#Cleaning the train_feature dataset
missing_data = train_features.isnull().sum()
missing_columns = missing_data[missing_data > 0].sort_values(ascending=False)

missing_columns


In [ ]:
#Before deciding on imputation strategies, 
#let's also check for missing values in the test_features dataset to ensure consistency in our approach across both datasets
missing_data = test_features.isnull().sum()
missing_columns = missing_data[missing_data > 0].sort_values(ascending=False)

missing_columns


In [ ]:
# Dividing the columns into both numerical and categorical

# Identify columns with missing values in train_features
missing_data_train = train_features.isnull().sum()
missing_columns_train = missing_data_train[missing_data_train > 0].index

# Categorize the missing columns into numerical and categorical types
numerical_columns = [col for col in missing_columns_train if train_features[col].dtype in ['int64', 'float64']]
categorical_columns = [col for col in missing_columns_train if train_features[col].dtype == 'object']

numerical_columns, categorical_columns

In [ ]:
# Dividing the columns into both numerical and categorical

# Identify columns with missing values in train_features
missing_data_test = test_features.isnull().sum()
missing_columns_test = missing_data_train[missing_data_train > 0].index

# Categorize the missing columns into numerical and categorical types
numerical_columns = [col for col in missing_columns_train if test_features[col].dtype in ['int64', 'float64']]
categorical_columns = [col for col in missing_columns_train if test_features[col].dtype == 'object']

numerical_columns, categorical_columns

In [ ]:
#We don't have any categorical values. We shall proceed with imputation strategies for the numerical columns

#1. We shall interpolate ndvi columns:
ndvi_columns = ['ndvi_ne', 'ndvi_nw', 'ndvi_se', 'ndvi_sw']
train_features[ndvi_columns] = train_features[ndvi_columns].interpolate()

#2. We shall impute the temperature and humidity-related columns using the median
temp_humidity_cols = ['station_avg_temp_c', 'reanalysis_air_temp_k', 'reanalysis_avg_temp_k',
                          'reanalysis_dew_point_temp_k', 'reanalysis_max_air_temp_k', 'reanalysis_min_air_temp_k',
                          'reanalysis_relative_humidity_percent', 'station_max_temp_c', 'station_min_temp_c',
                          'station_diur_temp_rng_c', 'reanalysis_tdtr_k', 'reanalysis_specific_humidity_g_per_kg']
for col in temp_humidity_cols:
    median_value = train_features[col].median()
    train_features[col].fillna(median_value, inplace=True)

#3.Fill missing values in precipitation columns with 0:
precipitation_cols = ['station_precip_mm', 'reanalysis_sat_precip_amt_mm', 'reanalysis_precip_amt_kg_per_m2','precipitation_amt_mm']
train_features[precipitation_cols] = train_features[precipitation_cols].fillna(0)

#4. Checking if we have any missing values now:
missing_after_imputation = train_features.isnull().sum().sum()
print(missing_after_imputation)


In [ ]:
#We don't have any categorical values. We shall proceed with imputation strategies for the numerical columns

#1. We shall interpolate ndvi columns:
ndvi_columns = ['ndvi_ne', 'ndvi_nw', 'ndvi_se', 'ndvi_sw']
test_features[ndvi_columns] = test_features[ndvi_columns].interpolate()

#2. We shall impute the temperature and humidity-related columns using the median
temp_humidity_cols = ['station_avg_temp_c', 'reanalysis_air_temp_k', 'reanalysis_avg_temp_k',
                          'reanalysis_dew_point_temp_k', 'reanalysis_max_air_temp_k', 'reanalysis_min_air_temp_k',
                          'reanalysis_relative_humidity_percent', 'station_max_temp_c', 'station_min_temp_c',
                          'station_diur_temp_rng_c', 'reanalysis_tdtr_k', 'reanalysis_specific_humidity_g_per_kg']
for col in temp_humidity_cols:
    median_value = test_features[col].median()
    test_features[col].fillna(median_value, inplace=True)

#3.Fill missing values in precipitation columns with 0:
precipitation_cols = ['station_precip_mm', 'reanalysis_sat_precip_amt_mm', 'reanalysis_precip_amt_kg_per_m2','precipitation_amt_mm']
test_features[precipitation_cols] = test_features[precipitation_cols].fillna(0)

#4. Checking if we have any missing values now:
missing_after_imputation = test_features.isnull().sum().sum()
print(missing_after_imputation)

In [ ]:
# As we are now done with missing values.Let's merge train_features with train_labels on ['city', 'year', 'weekofyear']
train_data = pd.merge(train_features, train_labels, on=['city', 'year', 'weekofyear'])

# Check the first few rows of the merged dataset
train_data.head()

In [ ]:
train_data.city.unique()

In [ ]:
train_data.shape,test_features.shape

In [ ]:
#Time Series Plot

train_data['approx_date'] = train_data['year'].astype(str) + '.' + train_data['weekofyear'].astype(str)

# Set the interval for showing x-ticks. We'll show every 50th tick for clarity.
interval = 50

# Plotting total_cases for each city
plt.figure(figsize=(20, 8))  # Increased the figure size
for city in ['sj', 'iq']:
    city_data = train_data[train_data['city'] == city]
    plt.plot(city_data['approx_date'], city_data['total_cases'], label=city)

# Customizing the plot
plt.title('Dengue Fever Cases Over Time')
plt.xlabel('Time (Year.WeekofYear)')
plt.ylabel('Total Cases')
plt.legend()
plt.xticks(rotation=90)  # Increased rotation for x-ticks

# Adjusting x-ticks for better visibility
ticks = plt.gca().get_xticks()
plt.gca().set_xticks(ticks[::interval])  # Show every 50th tick

plt.tight_layout()
plt.show()

Seasonality: Both cities show a clear seasonality in dengue cases, with peaks and troughs occurring at regular intervals.

Trend: San Juan (sj) seems to have a more noticeable upward trend in cases in the earlier years, which stabilizes later on. Iquitos (iq), on the other hand, has a relatively consistent number of cases across years, with some occasional spikes.

Differences Between Cities: San Juan generally has a higher number of cases compared to Iquitos. The patterns of dengue cases also differ between the two cities. This shows  the importance of considering city-specific factors when modeling.

In [ ]:
# Correlation Heatmap

# Exclude non-numeric columns and compute the correlation matrix
numeric_data = train_data.select_dtypes(include=['float64', 'int64'])
correlation_matrix_numeric = numeric_data.corr()

# Set up the matplotlib figure
plt.figure(figsize=(18, 14))

# Generate a heatmap for the numeric data
sns.heatmap(correlation_matrix_numeric, annot=True, cmap="coolwarm", linewidths=.5)

# Adjust the layout
plt.title('Correlation Heatmap for Numeric Features')
plt.tight_layout()
plt.show()

Positive Correlations with total_cases:

Some temperature and humidity metrics seem to have a moderate positive correlation with total_cases.


Negative Correlations with total_cases:

While there might be fewer strongly negative correlations in this dataset, it's essential to recognize any feature that might act inversely to dengue cases.

High Inter-feature Correlations:

Some features might be highly correlated with each other, indicating potential multicollinearity. This may impact the interpretability and stability of linear regression coefficients.
For instance, various temperature measurements seem to be correlated with each other. The same goes for some humidity metrics.

In [ ]:
# Features we want to visualize using boxplots
selected_features = ['reanalysis_air_temp_k', 'reanalysis_avg_temp_k', 'reanalysis_dew_point_temp_k',
                     'reanalysis_max_air_temp_k', 'reanalysis_min_air_temp_k', 'reanalysis_relative_humidity_percent',
                     'station_avg_temp_c', 'station_max_temp_c', 'station_min_temp_c']

# Plot boxplots for each of the selected features, separated by city
plt.figure(figsize=(20, 15))

for index, feature in enumerate(selected_features, 1):
    plt.subplot(3, 3, index)
    sns.boxplot(data=train_data, x='city', y=feature)
    plt.title(feature)

plt.tight_layout()
plt.show()



The box plots show that there are outliers that have to be dealt with

Feature Differences by City: Some features, like reanalysis_air_temp_k and reanalysis_avg_temp_k, show noticeable differences in their distributions between the two cities. This reinforces the idea that city-specific factors can significantly impact the data and modeling strategies should account for these differences.

In [ ]:
# Plotting histograms for selected features to understand their distributions
plt.figure(figsize=(20, 15))

for index, feature in enumerate(selected_features, 1):
    plt.subplot(3, 3, index)
    sns.histplot(train_data[feature], kde=True, bins=30)
    plt.title(f'Distribution of {feature}')

plt.tight_layout()
plt.show()


Variability: The spread of the data in the histograms provides insights into the variability of each feature.

Peaks: Multiple peaks in a distribution might suggest multiple sub-groups within the data or bimodal distributions.

Skewness: Some distributions show a bit of skewness either to the right (positive skew) or to the left (negative skew). Skewed distributions might sometimes benefit from transformations (like logarithmic or square root transformations) to make them more symmetric, especially for linear models that assume normally distributed errors.

KDE Overlay: The smooth line (Kernel Density Estimation) gives a smoothed version of the histogram and can help in identifying the distribution's mode(s).


**Feature Engineering**

In [ ]:
#1.Time-based Features. I'm trying to decide whether to convert the weeks  into months or quarters

# Extracting month from the week_start_date
train_data['month'] = pd.to_datetime(train_data['week_start_date']).dt.month

# Grouping by month and calculating the average number of cases
average_cases_by_month = train_data.groupby('month')['total_cases'].mean()

# Grouping by quarter and calculating the average number of cases
average_cases_by_quarter = train_data.groupby(train_data['month'] // 4 + 1)['total_cases'].mean()

# Plotting
plt.figure(figsize=(15, 6))

# Monthly average cases
plt.subplot(1, 2, 1)
average_cases_by_month.plot(kind='bar', color='skyblue')
plt.title('Average Dengue Cases by Month')
plt.xlabel('Month')
plt.ylabel('Average Cases')
plt.xticks(rotation=0)

# Quarterly average cases
plt.subplot(1, 2, 2)
average_cases_by_quarter.plot(kind='bar', color='salmon')
plt.title('Average Dengue Cases by Quarter')
plt.xlabel('Quarter')
plt.ylabel('Average Cases')
plt.xticks(rotation=0)

plt.tight_layout()
plt.show()

Months show finer distinctions. So I shall use months

In [ ]:
# Creating the month variables

# Convert month to a categorical variable
train_data['month'] = train_data['month'].astype('category')

# One-hot encode the month column
train_data = pd.get_dummies(train_data, columns=['month'], prefix='month', drop_first=True)

# Display the first few rows to show the one-hot encoded month columns
train_data.head()

In [ ]:

# Creating the month variables
test_features['month'] = pd.to_datetime(test_features['week_start_date']).dt.month
# Convert month to a categorical variable
test_features['month'] = test_features['month'].astype('category')

# One-hot encode the month column
test_features = pd.get_dummies(test_features, columns=['month'], prefix='month', drop_first=True)

# Display the first few rows to show the one-hot encoded month columns
test_features.head()


In [ ]:
# Creating interaction features
train_data['temp_precip_interaction'] = train_data['reanalysis_avg_temp_k'] * train_data['reanalysis_precip_amt_kg_per_m2']
train_data['temp_range_interaction'] = train_data['reanalysis_min_air_temp_k'] * train_data['reanalysis_max_air_temp_k']
train_data['dew_temp_interaction'] = train_data['reanalysis_dew_point_temp_k'] * train_data['reanalysis_avg_temp_k']

# Displaying the first few rows with the new interaction features
train_data[['reanalysis_avg_temp_k', 'reanalysis_precip_amt_kg_per_m2', 'temp_precip_interaction',
            'reanalysis_min_air_temp_k', 'reanalysis_max_air_temp_k', 'temp_range_interaction',
            'reanalysis_dew_point_temp_k', 'reanalysis_avg_temp_k', 'dew_temp_interaction']].head()


In [ ]:
# Creating interaction features
test_features['temp_precip_interaction'] = test_features['reanalysis_avg_temp_k'] * test_features['reanalysis_precip_amt_kg_per_m2']
test_features['temp_range_interaction'] = test_features['reanalysis_min_air_temp_k'] * test_features['reanalysis_max_air_temp_k']
test_features['dew_temp_interaction'] = test_features['reanalysis_dew_point_temp_k'] * test_features['reanalysis_avg_temp_k']

# Displaying the first few rows with the new interaction features
test_features[['reanalysis_avg_temp_k', 'reanalysis_precip_amt_kg_per_m2', 'temp_precip_interaction',
            'reanalysis_min_air_temp_k', 'reanalysis_max_air_temp_k', 'temp_range_interaction',
            'reanalysis_dew_point_temp_k', 'reanalysis_avg_temp_k', 'dew_temp_interaction']].head()


In [ ]:
temp_humidity_cols.extend(['temp_precip_interaction','temp_range_interaction','dew_temp_interaction'])

# Creating 4-week lags for the temp_humidity_cols in train dataset
for column in temp_humidity_cols:
    train_data[f'{column}_lag4'] = train_data.groupby('city')[column].shift(4)

# Display the first few rows with the new lagged features
train_data[['city', 'year', 'weekofyear'] + [f'{column}_lag4' for column in temp_humidity_cols ]].head()


In [ ]:
# Creating 4-week lags for the temp_humidity_cols in the test dataset
for column in temp_humidity_cols:
    test_features[f'{column}_lag4'] = test_features.groupby('city')[column].shift(4)

# Display the first few rows with the new lagged features
test_features[['city', 'year', 'weekofyear'] + [f'{column}_lag4' for column in temp_humidity_cols]].head(10)

In [ ]:
# Function to handle outliers using the IQR method
def handle_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Capping the outliers
    data[column] = data[column].apply(lambda x: upper_bound if x > upper_bound else (lower_bound if x < lower_bound else x))
    
    return data

# Handling outliers for the selected features
for feature in selected_features:
    train_data = handle_outliers_iqr(train_data, feature)

# Displaying the cleaned data
train_data[selected_features].describe()


In [ ]:
# Function to handle outliers using the IQR method
def handle_outliers_iqr(data, column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    # Capping the outliers
    data[column] = data[column].apply(lambda x: upper_bound if x > upper_bound else (lower_bound if x < lower_bound else x))
    
    return data

# Handling outliers for the selected features
for feature in selected_features:
   test_features = handle_outliers_iqr(test_features, feature)

# Displaying the cleaned data
test_features[selected_features].describe()

**Feature Selection**

In [ ]:
# Filter the dataset to exclude non-numerical columns
numerical_data = train_data.select_dtypes(exclude=['object'])

# Calculate the correlation of each numerical feature with the target variable 'total_cases'
numerical_correlations = numerical_data.corr()['total_cases'].sort_values(ascending=False)

# Display the correlations
numerical_correlations


In [ ]:
# Selecting features with an absolute correlation value greater than 0.1
selected_features = numerical_correlations[abs(numerical_correlations) > 0.1].index.tolist()

# Excluding the target variable 'total_cases' from the selected features list
selected_features.remove('total_cases')

# Displaying the selected features
selected_features


Feature selection by Random Forest

In [ ]:
# Count the rows with NaN values in the lagged columns
nan_values = train_data.isnull().sum()

nan_values

In [ ]:
train_data = train_data.dropna()

In [ ]:
train_data.shape, test_features.shape

In [ ]:
# Adjusting the columns list to drop and then proceeding
columns_to_drop = ['city', 'year', 'weekofyear', 'week_start_date', 'total_cases']

X = train_data.drop(columns_to_drop, axis=1)
y = train_data['total_cases']

# Initializing the random forest regressor
rf = RandomForestRegressor(n_estimators=100, random_state=42)
# Training the random forest regressor
rf.fit(X, y)

# Extracting feature importances and creating a DataFrame for visualization
feature_importances = pd.DataFrame({
    'feature': X.columns,
    'importance': rf.feature_importances_
})

# Sorting the DataFrame by importance
feature_importances = feature_importances.sort_values(by='importance', ascending=False)

# Displaying the top 20 features
top_features = feature_importances.head(25)

top_features


In [ ]:
#Take the intersection of the top features from both the correlation method and the Random Forest feature importance.
top_features_list = top_features['feature'].tolist()

common_features = [feature for feature in selected_features if feature in top_features_list]
common_features


In [ ]:
#Selecting the top features for both the train_data and test_features

train_data = train_data[common_features + ['total_cases', 'city', 'year', 'weekofyear']]
test_features = test_features[common_features + ['city', 'year', 'weekofyear']]

print(train_data.shape, test_features.shape)



**Split the data into training and validation sets**

In [ ]:
from sklearn.model_selection import train_test_split


# Create the validation sets
X = train_data.drop(columns=['total_cases'])
y = train_data['total_cases']


# Splitting data into training and validation sets (80% train, 20% validation)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)




In [ ]:
# One-hot encoding the 'city' variable for both training and validation sets
X_train = pd.get_dummies(X_train, columns=['city'], drop_first=True)
X_val = pd.get_dummies(X_val, columns=['city'], drop_first=True)

X_train.head()
print(X_train.columns)

**Feature Scaling**

In [ ]:
from sklearn.preprocessing import RobustScaler

# Initialize the RobustScaler
robust_scaler = RobustScaler()

# Fit and transform the training data using the robust scaler
X_train_scaled = robust_scaler.fit_transform(X_train)

# Transform the validation data using the robust scaler
X_val_scaled = robust_scaler.transform(X_val)

X_train_scaled[:5]  # Display the first 5 rows of the scaled training data

**Feature Modeling**

In [ ]:
#1. Linear Regression
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

# Initialize the linear regression model
lin_reg = LinearRegression()

# Train the model using the scaled training data
lin_reg.fit(X_train_scaled, y_train)

# Predict on the validation set
y_val_pred = lin_reg.predict(X_val_scaled)

# Calculate the mean absolute error
mae_linear_regression = mean_absolute_error(y_val, y_val_pred)
mae_linear_regression

In [ ]:
#2. Ridge Regression
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error

# Initialize Ridge Regression model
ridge_model = Ridge(alpha=1.0)  # You can adjust the alpha parameter for regularization strength

# Train the model
ridge_model.fit(X_train_scaled, y_train)

# Predict on the validation set
y_pred_ridge = ridge_model.predict(X_val_scaled)

# Calculate the mean absolute error
mae_ridge = mean_absolute_error(y_val, y_pred_ridge)
print(f"Mean Absolute Error (Ridge Regression): {mae_ridge}")

In [ ]:
#3. Decision Tree:
from sklearn.tree import DecisionTreeRegressor

# Initialize the Decision Tree model
tree_model = DecisionTreeRegressor(max_depth=10)  # Adjust max_depth for tree depth

# Train the model
tree_model.fit(X_train_scaled, y_train)

# Predict on the validation set
y_pred_tree = tree_model.predict(X_val_scaled)

# Calculate the mean absolute error
mae_tree = mean_absolute_error(y_val, y_pred_tree)
print(f"Mean Absolute Error (Decision Trees): {mae_tree}")

In [ ]:
#4. Random Forest

from sklearn.ensemble import RandomForestRegressor

# Initialize the Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)

# Train the model
rf_model.fit(X_train_scaled, y_train)

# Predict on the validation set
y_pred_rf = rf_model.predict(X_val_scaled)

# Calculate the mean absolute error
mae_rf = mean_absolute_error(y_val, y_pred_rf)
print(f"Mean Absolute Error (Random Forest): {mae_rf}")


In [ ]:
#Hyper parameter Optimization for Gradient Boosting Regressor

gb = GradientBoostingRegressor(random_state=42)

# Define hyperparameters and their possible values
param_grid_gb = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 0.9, 1.0],
    'max_features': ['sqrt', 'log2', None]
}

# Use TimeSeriesSplit for cross-validation
tscv = TimeSeriesSplit(n_splits=5)

# Set up GridSearchCV for Gradient Boosting
grid_search_gb = GridSearchCV(gb, param_grid_gb, cv=tscv, scoring='neg_mean_absolute_error', n_jobs=-1, verbose=1)

# Fit to the data
grid_search_gb.fit(X_train_scaled, y_train)

# Extract the best hyperparameters
best_params_gb = grid_search_gb.best_params_

In [ ]:
# Use the best hyperparameters from the grid search for Gradient Boosting
best_gb_params = grid_search_gb.best_params_

# Initialize the Gradient Boosting model with the best hyperparameters
gb_model = GradientBoostingRegressor(**best_gb_params, random_state=42)

# Train the model
gb_model.fit(X_train_scaled, y_train)

# Predict on the validation set
y_pred_gb = gb_model.predict(X_val_scaled)

# Calculate the mean absolute error
mae_gb = mean_absolute_error(y_val, y_pred_gb)
print(f"Mean Absolute Error (Gradient Boosting): {mae_gb}")



In [ ]:
 #Hyper parameter Optimizationfor XGBoost

xgb_regressor = xgb.XGBRegressor(random_state=42)

# Define hyperparameters and their possible values
param_grid_xgb = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.05, 0.1],
    'gamma': [0, 0.1, 0.2],
    'subsample': [0.7, 0.8, 0.9],
    'colsample_bytree': [0.7, 0.8, 0.9],
    'max_depth': [5, 6, 7]
}

# Set up GridSearchCV for XGBoost
grid_search_xgb = GridSearchCV(xgb_regressor, param_grid_xgb, cv=tscv, scoring='neg_mean_absolute_error', n_jobs=-1, verbose=1)

# Fit to the data
grid_search_xgb.fit(X_train_scaled, y_train)

# Extract the best hyperparameters
best_params_xgb = grid_search_xgb.best_params_


In [ ]:

# Using the best hyperparameters from the grid search for XGBoost
best_xgb_params = grid_search_xgb.best_params_

# Initialize the XGBRegressor with the best hyperparameters
xgb_regressor = xgb.XGBRegressor(**best_xgb_params, random_state=42)

# Train the model
xgb_regressor.fit(X_train_scaled, y_train)

# Predict on the validation set
y_pred_xgb = xgb_regressor.predict(X_val_scaled)

# Calculate the mean absolute error for XGBoost
mae_xgb = mean_absolute_error(y_val, y_pred_xgb)
print(f"Mean Absolute Error (XGBoost): {mae_xgb}")



We will use a **Stacked Model of Xgboost and Gradient Boosting** as they both have the **lowest MAE**

**Feature Engineering the Test Data and creating Predictions**

In [ ]:

# Handling NaN values in the test features
nan_columns = test_features.columns[test_features.isnull().any()]
for col in nan_columns:
    median_value = train_data[col].median()  # Using median from training data to avoid data leakage
    test_features[col].fillna(median_value, inplace=True)

# One-hot encode city column and dropping total_cases 
test_features = pd.get_dummies(test_features, columns=['city'], drop_first=True) 
test_features = test_features.drop('total_cases', axis=1, errors='ignore') 

# Scaling test features
test_scaled = robust_scaler.transform(test_features)

# Predicting on the validation set using Gradient Boosting and XGBoost
y_pred_gb = gb_model.predict(X_val_scaled)
y_pred_xgb = xgb_regressor.predict(X_val_scaled)

# Stacking the predictions from the base models to form a new training set for the meta-model
stacked_predictions = np.column_stack((y_pred_gb, y_pred_xgb))

# Initializing and training the meta-model
meta_model = LinearRegression()
meta_model.fit(stacked_predictions, y_val)

# Predicting using individual models on the test set
gb_test_preds = gb_model.predict(test_scaled)
xgb_test_preds = xgb_regressor.predict(test_scaled)

# Stacking the test set predictions
stacked_test_predictions = np.column_stack((gb_test_preds, xgb_test_preds))

# Use the meta-model to make final predictions on the test set
final_predictions = meta_model.predict(stacked_test_predictions)
final_predictions = final_predictions.round().astype(int)


In [ ]:

test_features['city'] = np.where(test_features['city_sj'] == 1, 'sj', 'iq')

submission = pd.DataFrame({
        "city": test_features.city,
        "year": test_features.year,
    "weekofyear":test_features.weekofyear,
    "total_cases":final_predictions
    })
submission
submission.to_csv('submission_stacked.csv', index=False)